### This notebook scrapes some quick info from 2100 highest rated restaurant in GTA from tripAdvisor 

In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
%run scrape_googlemaps_functions.ipynb
from datetime import datetime
import pytz
import pandas as pd

In [2]:
from selenium import webdriver
# options = Options()
# options.add_argument("--lang=en")
# driver = webdriver.Chrome(chrome_options=options)

driver = webdriver.Chrome(executable_path=r'C:\Users\hajih\Downloads\chromedriver_win32\chromedriver.exe')

In [3]:
# Open Table Toronto restaurants
# url = 'https://www.opentable.ca/s?dateTime=2021-08-09T19%3A00%3A00&covers=2&latitude=43.651007&longitude=-79.420648&corrid=08e0949d-fbe7-428a-968d-c988e1d101e3&sortBy=rating&metroId=74&queryUnderstandingType=none'
url = 'https://www.tripadvisor.ca/Restaurants-g155019-Toronto_Ontario.html'

x = driver.get(url)

In [4]:
def go_to_page(driver,p):
    # p = 1 goes to page 2 and so on
    links = driver.find_elements_by_xpath(f'//*[@id="EATERY_LIST_CONTENTS"]/div[2]/div/div/a[{p}]') # has to change to match stucture
    #     print('LINKS HERE', links)
    for l in links:
        l.click()
    time.sleep(5)       


In [5]:
# if we go over 30 pages
soup = BeautifulSoup(driver.page_source,'html.parser')
names = soup.find_all('a', class_='_15_ydu6b')
stars =  soup.find_all('div', class_='_2rmp5aUK')
cuisine = soup.find_all('div',class_='MIajtJFg _1cBs8huC _3d9EnJpt')

rest_names = []
for rest in names:
    rest_names.append(rest.text)

rest_stars = []
for rest in stars:
    rest_stars.append(rest.find('svg',class_='zWXXYhVR')['aria-label'])

rest_cuisine = []
retdates = []

for rest in cuisine:
    rest_cuisine.append(rest.find('span',class_="_1p0FLy4t").text)
    t        = datetime.now()
    timezone = pytz.timezone('Canada/Eastern')
    t        = timezone.localize(t)
    retdates.append(t)


In [6]:
import time
for p in range(1,6):
    print(p+1)
    go_to_page(driver,p)
    soup = BeautifulSoup(driver.page_source,'html.parser')
    names = soup.find_all('a', class_='_15_ydu6b')
    stars =  soup.find_all('div', class_='_2rmp5aUK')
    cuisine = soup.find_all('div',class_='MIajtJFg _1cBs8huC _3d9EnJpt')
    for rest in names:
        rest_names.append(rest.text)
        
    for rest in stars:
        rest_stars.append(rest.find('svg',class_='zWXXYhVR')['aria-label'])

    for rest in cuisine:
        rest_cuisine.append(rest.find('span',class_="_1p0FLy4t").text)
        t        = datetime.now()
        timezone = pytz.timezone('Canada/Eastern')
        t        = timezone.localize(t)
        retdates.append(t)


2
3
4
5
6


In [7]:
max_pages = 70
page = 6
while page < max_pages:
    print(page)
    page +=1 
# for p in range(1,6):
    go_to_page(driver,5) # next page is always coded as 5
    soup = BeautifulSoup(driver.page_source,'html.parser')
    names = soup.find_all('a', class_='_15_ydu6b')
    stars =  soup.find_all('div', class_='_2rmp5aUK')
    cuisine = soup.find_all('div',class_='MIajtJFg _1cBs8huC _3d9EnJpt')
    for rest in names:
        rest_names.append(rest.text)
        
    for rest in stars:
        try:
            rest_stars.append(rest.find('svg',class_='zWXXYhVR')['aria-label'])
        except Exception as e:
            rest_stars.append('None')
            
    for rest in cuisine:
        try:
            rest_cuisine.append(rest.find('span',class_="_1p0FLy4t").text)
        except Exception as e:
            rest_cuisine.append('None') 

        t        = datetime.now()
        timezone = pytz.timezone('Canada/Eastern')
        t        = timezone.localize(t)
        retdates.append(t)


6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69


In [11]:
rest_dict = dict()
rest_dict['name'] = rest_names
rest_dict['stars'] = rest_stars
rest_dict['cuisine'] = rest_cuisine
rest_dict['retrieve-date'] = retdates

restaurant_df = pd.DataFrame.from_dict(rest_dict)
csvfile = 'data/restaurant_list_Toronto_tripAdvisor.csv'
restaurant_df.to_csv(csvfile)
